### Model
| model |
| --- |
| gpt-3.5-turbo-0125 |
| gpt-4-0125-preview |

In [4]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1
)

In [6]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in {language}"),
    AIMessage(content="Hi my name is {name}!"),
    HumanMessage(content="서울과 도쿄의 거리는 얼마인가요. 너의 이름은 뭐니?"),
]

chat.predict_messages(messages)

AIMessage(content='서울과 도쿄의 직선 거리는 약 1,200km 정도입니다. 제 이름은 루카스입니다.')

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1
)

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Hi my name is {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
    ]
)

prompt = template.format_messages(
    language = "Korean",
    name = "AiGENDA",
    country_a = "Korea",
    country_b = "USA"
)

chat.predict_messages(prompt).content

'한국과 미국 사이의 거리는 대략 10,000km입니다. 제 이름은 AiGENDA입니다.'

## 3.3 OutputParser and LCEL 

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        
        return list(map(str.strip, items))

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0
)

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else. And you only reply in {language}."),
        ("ai", "Hi my name is {name}!"),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(
    name = "aigenda",
    language = "Korean",
    max_items = 10,
    question = "2024 한국 축구 국가대표 맴버",
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['1. 손흥민',
 '2. 황의조',
 '3. 김민재',
 '4. 김영권',
 '5. 김민규',
 '6. 김진수',
 '7. 김영광',
 '8. 이재성',
 '9. 황인범',
 '10. 김승대']

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else. And you only reply in {language}."),
        ("ai", "Hi my name is {name}!"),
        ("human", "{question}"),
    ]
)

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0
)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        
        return list(map(str.strip, items))

chain = template | chat | CommaOutputParser()    

chain.invoke({
    "name":"aigenda",
    "language":"Korean",
    "max_items":5,
    "question":"2024 한국 축구 국가대표 맴버",
})

['손흥민', '황의조', '김민재', '김영권', '김민재']

## 3.4 Chaining Chains

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. \
         Everything you are asked will be answered with a comma separated list of max 5 in lowercase. \
         Do NOT reply with anything else."),
        # ("human", "{question}"),
        ("human", "I want to cook {cuisine} food."),
    ]
)

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. \
         You find alternative ingredients and explain their preparation. \
         You don't radically modify the recipe. \
         If there is no alternative for a food just say you don't know how to replace it. \
         And you only reply in Korean"),
        ("human", "{recipe}")
    ]
)

chef_chain = chef_prompt | chat

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine":"korea"})



kimchi, bulgogi, bibimbap, tteokbokki김치는 대체할 수 있는 재료가 없습니다. 

불고기를 채식주의자용으로 만들려면 대체 고기로 대신할 수 있습니다. 대체 고기로는 대부분 식물성 단백질이나 버섯을 사용합니다. 대체 고기를 사용할 때는 불고기 소스와 함께 볶아내면 됩니다.

비빔밥은 채소나 과일을 활용하여 대체할 수 있습니다. 대체 재료로는 아보카도, 콩, 토마토, 당근, 오이 등을 사용할 수 있습니다. 이를 채소와 함께 밥 위에 올려 비빔밥 소스와 함께 섞어먹으면 됩니다.

떡볶이는 떡 대신에 떡 대신에 고구마, 양배추, 어묵, 버섯 등을 사용하여 대체할 수 있습니다. 떡 대신에 이러한 채소나 버섯을 썰어서 떡볶이 소스와 함께 볶아내면 됩니다.

AIMessageChunk(content='김치는 대체할 수 있는 재료가 없습니다. \n\n불고기를 채식주의자용으로 만들려면 대체 고기로 대신할 수 있습니다. 대체 고기로는 대부분 식물성 단백질이나 버섯을 사용합니다. 대체 고기를 사용할 때는 불고기 소스와 함께 볶아내면 됩니다.\n\n비빔밥은 채소나 과일을 활용하여 대체할 수 있습니다. 대체 재료로는 아보카도, 콩, 토마토, 당근, 오이 등을 사용할 수 있습니다. 이를 채소와 함께 밥 위에 올려 비빔밥 소스와 함께 섞어먹으면 됩니다.\n\n떡볶이는 떡 대신에 떡 대신에 고구마, 양배추, 어묵, 버섯 등을 사용하여 대체할 수 있습니다. 떡 대신에 이러한 채소나 버섯을 썰어서 떡볶이 소스와 함께 볶아내면 됩니다.')